# A cautionary tale of ground state energy calculation

When calculating the ground state energy of an electronic structure Hamiltonian, some care should be exercised. Converting the Hamiltonian to a sparse matrix and finding the least eigenvalue will not always be the _true_ ground state with respect to the system it represents, since the diagonalization scheme used may not inherently observe any symmetries present in the underlying physical system.

Consider the following $H_3^+$ example, consisting of 2 electrons in 6 spin-orbitals...

In [1]:
from symmer.chemistry import MoleculeBuilder
from symmer.symplectic import QuantumState

basis='sto-3g'
charge=+1
geometry =[
    ("H", [0., 0.558243000, 0.]),
    ("H", [0.483452000, -0.279121000, 0.]),
    ("H", [-0.483452000, -0.279121000, 0.])
]
molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True)

Molecule geometry:
H	0.0	0.558243	0.0
H	0.483452	-0.279121	0.0
H	-0.483452	-0.279121	0.0

CISD converged?  True
FCI converged?  True
FCI converged?  True

HF energy:   -1.2468600063384454
MP2 energy:  -1.2658602663569558
CCSD energy: -1.274144616958312
CISD energy: -1.274144467123986
FCI energy:  -1.2741444671239879


Number of qubits: 6


Naively computing the ground state energy by taking the smallest eigenvalue of the Hamiltonian does not match the FCI energy:

In [2]:
from symmer.utils import exact_gs_energy

gs_nrg, gs_vec = exact_gs_energy(molecule.H_q.to_sparse_matrix)

print(f'Least eigenvalue = {gs_nrg} | FCI error = {gs_nrg - molecule.fci_energy}')

Least eigenvalue = -1.3507306438907747 | FCI error = -0.0765861767667868


What has gone wrong here? Taking a look at the corresponding eigenvector, we see the identified state actually contains 3 particles, whereas the underlying system only contains 2:

In [3]:
psi = QuantumState.from_array(gs_vec).cleanup(zero_threshold=1e-5)
print(f'Eigenvector with eigenvalue {gs_nrg}:\n')
print(psi)

Eigenvector with eigenvalue -1.3507306438907747:

-0.063-0.102j |001011> +
-0.076-0.123j |100011> +
 0.076+0.123j |101100> +
 0.508+0.828j |111000>


To counter this issue, we instead need to select the least eigenvalue that contains the _correct_ number of particles. This is implemented in `symmer.chemistry.exact_gs_energy`

In [4]:
gs_nrg, gs_vec = exact_gs_energy(
    molecule.H_q.to_sparse_matrix, 
    n_particles=molecule.n_particles, 
    number_operator=molecule.number_operator
)
psi = QuantumState.from_array(gs_vec).cleanup(zero_threshold=1e-5)

print(f'Least eigenvalue = {gs_nrg} | FCI error = {gs_nrg - molecule.fci_energy}\n')
print(f'Eigenvector with eigenvalue {gs_nrg}:\n')
print(psi)

Least eigenvalue = -1.274144467123995 | FCI error = -7.105427357601002e-15

Eigenvector with eigenvalue -1.274144467123995:

 0.082+0.045j |000011> +
 0.082+0.045j |001100> +
-0.867-0.479j |110000>


Success! We have now recovered the true ground state by enforcing that only solutions with the correct number of particles are considered. Note however it is possible that no solution is identified at first - in this case, increase the `n_eigs` parameter in `symmer.chem.exact_gs_state` to increase the search space.